In [ ]:
# BrightEarth AI Refactored Notebook

# Cell 1: Imports & Secure Setup
import os
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.credentials import Credentials

# Load sensitive credentials from environment variables
API_KEY = os.getenv("WATSONX_API_KEY","ch1nbczDMcqQRSQY5EeWxvGL5R_DfCig43bnKxxfB58w")
PROJECT_ID = os.getenv("WATSONX_PROJECT_ID","ae528503-ba43-464b-8eb2-add4c13852ce")
REGION = os.getenv("WATSONX_REGION", "us-south")

if not API_KEY or not PROJECT_ID:
    raise EnvironmentError("Please set WATSONX_API_KEY and WATSONX_PROJECT_ID environment variables.")

# Initialize IBM WatsonX Granite model inference
creds = Credentials(
    url=f"https://{REGION}.ml.cloud.ibm.com",
    api_key=API_KEY
)
model = ModelInference(
    model_id="ibm/granite-3-8b-instruct",
    credentials=creds,
    project_id=PROJECT_ID
)

# Cell 2: Central Inference Utility
def run_inference(
    prompt: str,
    max_new_tokens: int = 80,
    temperature: float = 0.7,
    top_p: float = 0.9,
    frequency_penalty: float = 0.0,
    presence_penalty: float = 0.0,
    stop_sequences: list = None
) -> str:
    """
    Utility to send prompts to the Granite model and return generated text.
    """
    params = {
        "max_new_tokens": max_new_tokens,
        "temperature": temperature,
        "top_p": top_p,
        "frequency_penalty": frequency_penalty,
        "presence_penalty": presence_penalty,
    }
    if stop_sequences:
        params["stop_sequences"] = stop_sequences

    response = model.generate(prompt=prompt, params=params)
    return response["results"][0]["generated_text"].strip()

# Cell 3: Eco-Action Tip Generator
INSTRUCTION = (
    "You are BrightEarth AI, a sustainability co-pilot.\n"
    "If role, environment, or task is missing, ask for them.\n"
    "Once all are provided, return ONE eco-action tip (<40 words), human-sounding, realistic, aligned to UN SDG 8.\n"
    "Never mention user info. Never output more than one tip. Do not use titles or citations. Only reply with the tip."
)

def generate_tip(job_role: str, work_env: str, current_task: str) -> str:
    if not (job_role and work_env and current_task):
        return "Can you remind me of your job role, work environment, and current task?"
    prompt = (
        f"{INSTRUCTION}\n\n"
        f"Role={job_role}; Environment={work_env}; Task={current_task}\n"
    )
    return run_inference(
        prompt,
        max_new_tokens=80,
        temperature=0.7,
        top_p=0.9,
        frequency_penalty=0.2,
        presence_penalty=0.2,
        stop_sequences=["<|endoftext|>"]
    )

# Cell 4: Summarizer Function
def summarize_text(text: str) -> str:
    if not text.strip():
        return "Please provide text to summarize."
    prompt = f"Summarize the following text into two clear, professional sentences:\n\n{text}\n"
    return run_inference(prompt, max_new_tokens=60, temperature=0.5)

# Cell 5: Professional Report Writer
def generate_report(topic: str) -> str:
    if not topic.strip():
        return "Please provide a topic for the report."
    prompt = (
        f"Draft a concise professional report on '{topic}'."
        " Include an introduction, key points, and a conclusion."
    )
    return run_inference(prompt, max_new_tokens=1000, temperature=0.7)

# Cell 6: Innovation Trend Finder
def innovation_trend(job_role: str) -> str:
    if not job_role.strip():
        return "Please provide a job role to discover relevant trends."
    prompt = (
        f"What is one cutting-edge innovation or technology trend that a {job_role} should know about?"
        " Provide one actionable insight."
    )
    return run_inference(prompt, max_new_tokens=60, temperature=0.7)

# Cell 7: Vision Model Setup
vision_model = ModelInference(
    model_id="ibm/granite-vision-2b-vision-instruct",
    credentials=creds,
    project_id=PROJECT_ID
)

# Cell 8: Eco-Action Proof Verifier
def verify_proof(image_path: str) -> str:
    """
    Uses Granite Vision to verify and describe an eco-action proof image.
    Returns a brief description or classification.
    """
    with open(image_path, "rb") as f:
        img_bytes = f.read()
    prompt = "Verify this image as valid proof of an eco-action by describing what it shows."
    response = vision_model.generate(
        prompt=prompt,
        inputs={"image": img_bytes},
        params={"max_new_tokens": 100, "temperature": 0.5}
    )
    return response["results"][0]["generated_text"].strip()

# Example usage (uncomment and replace path):
# print(verify_proof("path/to/recycling_photo.png"))
